In [1]:
# Verify we're in the correct working directory
import os
os.getcwd()   

'/Users/seraphinashi/Desktop/l3c_ctml'

In [2]:
# pip install -r requirements.txt

In [3]:
from src.STEP1_feature import a_cohort as a
from src.STEP1_feature import b_medication as b
import os
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType

In [ ]:
spark = SparkSession.builder \
        .master("local[1]") \
        .appName("l3c_ctml") \
        .getOrCreate()

# Read CSV file into table
data_path = "synthetic_data/training"

schema = StructType([
    StructField('table', StringType(), True),
    StructField('field', StringType(), True),
    StructField('type', StringType(), True)
])

all_fields = spark.createDataFrame([], schema)
    
for path, subdirs, files in os.walk(data_path):
    for name in files:
        if "csv" in name and name[0]!=".":
            # table_fields = [(table_name, f.name, str(f.dataType)) for f in datafile.schema.fields]
            full_file = (os.path.join(path, name))
            df = spark.read.csv(os.path.join(path, name), header=True, inferSchema=True)
            var_name = name[:-4]
            print(full_file)
            globals()[var_name] = df

synthetic_data/training/observation_period.csv
synthetic_data/training/drug_exposure.csv
synthetic_data/training/death.csv
synthetic_data/training/concept.csv
synthetic_data/training/measurement.csv
synthetic_data/training/condition_occurrence.csv


In [ ]:
# 1_cohort_and_features

covid_pasc_index_dates = a.sql_statement_01(long_covid_silver_standard)
cohort = a.sql_statement_00(covid_pasc_index_dates, person)
long_covid_patients = a.sql_statement_08(covid_pasc_index_dates)

hosp_cases = a.sql_statement_04(cohort, condition_occurrence, microvisits_to_macrovisits)

hosp_and_non = a.sql_statement_06(cohort, hosp_cases)

Feature_Table_Builder_v0 = a.sql_statement_03(covid_pasc_index_dates, hosp_and_non, microvisits_to_macrovisits)
icu_visits = a.sql_statement_05(microvisits_to_macrovisits, concept) # empty for some reason
inpatient_visits = a.sql_statement_07(microvisits_to_macrovisits, concept)

tot_icu_days_calc = a.sql_statement_09(Feature_Table_Builder_v0, icu_visits)
tot_ip_days_calc = a.sql_statement_10(Feature_Table_Builder_v0, inpatient_visits)

Feature_Table_Builder = a.sql_statement_02(Feature_Table_Builder_v0, tot_icu_days_calc, tot_ip_days_calc)

In [ ]:
# 2_med_feature_table
DrugConcepts = b.sql_statement_00(concept)
Drugs_for_These_Patients = b.sql_statement_01(Feature_Table_Builder, drug_exposure)

drugRollUp = b.sql_statement_04(DrugConcepts, Drugs_for_These_Patients)

covid_drugs = b.sql_statement_02(Feature_Table_Builder, drugRollUp)
pre_pre_drugs = b.sql_statement_10(Feature_Table_Builder, drugRollUp)
pre_drugs = b.sql_statement_07(Feature_Table_Builder, drugRollUp)
post_drugs = b.sql_statement_05(Feature_Table_Builder, drugRollUp)

covidtbl = b.sql_statement_03(Feature_Table_Builder, covid_drugs, microvisits_to_macrovisits)
prepretbl = b.sql_statement_11(Feature_Table_Builder, covid_drugs, microvisits_to_macrovisits)
pretbl = b.sql_statement_12(Feature_Table_Builder, pre_pre_drugs, microvisits_to_macrovisits)
posttbl = b.sql_statement_06(Feature_Table_Builder, post_drugs, microvisits_to_macrovisits)

pre_post_med_count = b.sql_statement_08(covidtbl, posttbl, prepretbl, pretbl)

pre_post_med_count_clean = b.sql_statement_09(Feature_Table_Builder, pre_post_med_count)